# TC1- JUEGO INTELIGENTE: ESQUINAS CHINAS
## Implementación con estrategia Primero el mejor

#### Integrantes del Grupo Nº1
| NOMBRES Y APELLIDOS | CODIGO |ROL|
|:----------|----------|:----------|
| Morán Huamán, Dajanna| 19200181 | Integrante|
| Ortiz Abanto Kevin Miguel| 20200230 | Integrante|
| Pachas Lopez Nabia Jazmin| 20200231 | Integrante| 
| Retuerto Santillan Mauro Alejandro| 20200202 | Coordinador|

## DESARROLLO DEL CODIGO
### Parte Inicial

**Importación de Bibliotecas:** Para la codificación del juego inteligente se utilizaron  dos bibliotecas: *math* para las funciones matematicas, y *copy* para la utilización del método `deepcopy()`.

In [270]:
import math as mt
import copy as cp

**Variables Globales:** Entre las variables globales que se definieron, se tiene:
- `T`  → Tablero actual.

- `T_Meta_M` → Tablero meta para la máquina.

- `T_Meta_H` → Tablero meta para el oponente humano.

- `pos` → Array de posiciones actuales de las fichas de la máquina.

- `pos_Meta_M`	→ Array de posiciones metas para las fichas de la máquina.

- `pos_prom_Meta` → Posicion meta promedio para las fichas de la máquina.

- `n_fichas` → Cantidad de fichas

- `n_movs_M` → Cantidad de movimientos realizados por la máquina.

- `ficha_mov` → Última ficha movida en el juego

In [271]:
# Definiendo las variables globales
T = []
T_Meta_M = []
T_Meta_H = []
pos = []
pos_Meta_M = []
pos_prom_Meta = []
n_fichas = 6
n_movs_M = 0
ficha_mov = -1
movs = {}

**Funcion `pos_promedio()`:** Recibe como parámetro a `pos_fichas` que es un array de las posiciones de las 6 fichas del jugador. Esta función calcula la posición promedio de las 6 fichas (también llamado baricentro) y lo retorna.

In [272]:
def pos_promedio(pos_fichas):
    sumx = sumy = 0.0
    for i in range(len(pos_fichas)):
        sumx = sumx + pos_fichas[i][0]
        sumy = sumy + pos_fichas[i][1]
    promx = sumx / n_fichas
    promy = sumy / n_fichas
    prom = [promx, promy]
    return prom

**Funcion `obtener_pos()`:** Recibe como parámetros a `tab`, que es el tablero actual, y a `jugador`, que es el valor númerico que representa al jugador (para máquina es 1, para jugador es -1 y posición vacía es 0). A partir de ello, esta función crea el array de posiciones de un jugador específico. Para ello recorre el tablero comparando los elementos.

In [273]:
def obtener_pos(tab, jugador):
    p = []
    
    for i in range(len(tab)):
        
        for j in range(len(tab[i])):
            
            if tab[i][j] == jugador:
                posicion = [i, j]
                p.append(posicion)
    return p

**Función `estado_inicial()`:** Esta función inicializa el valor o contenido de las variables globales.

In [274]:
def estado_inicial():
    global T, T_Meta_M, T_Meta_H

    T = [[1, 1, 1, 0, 0, 0],
         [1, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0,-1],
         [0, 0, 0, 0,-1,-1],
         [0, 0, 0,-1,-1,-1]
         ]

    T_Meta_M = [[0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 1, 1],
                [0, 0, 0, 1, 1, 1]
                ]

    T_Meta_H = [[-1,-1,-1, 0, 0, 0],
                [-1,-1, 0, 0, 0, 0],
                [-1, 0, 0, 0, 0, 0],
                [ 0, 0, 0, 0, 0, 0],
                [ 0, 0, 0, 0, 0, 0],
                [ 0, 0, 0, 0, 0, 0]
                ]
    global pos, pos_Meta_M
    pos = [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [2, 0]]
    pos_Meta_M = [[3, 5], [4, 4], [4, 5], [5, 3], [5, 4], [5, 5]]

    global movs
    movs = {
        1: (-1, 0),
        2: (1, 0),
        3: (0, -1),
        4: (0, 1),
        5: (-1, -1),
        6: (1, 1),
        7: (-1, 1),
        8: (1, -1)
    }
    global pos_prom_Meta
    pos_prom_Meta = pos_promedio(pos_Meta_M)

**Funcion `codificar()`:** Recibe un valor númerico `v` y lo codifica en un símbolo/letra. En este caso, se decidió que las fichas de la máquina se almacenran en el tablero como 1 pero que se muestren en pantalla como X; y que las fichas del humano se almacenran en el tablero como -1 pero que se muestren en pantalla como O.

In [275]:
def codificar(v):
    if v == -1:
        return 'O'
    elif v == 1:
        return 'X'
    elif v == 0:
        return "-"

**Funcion `mostrar_tablero()`:** Esta función recibe una matriz .tab.,y la muestra en consola. Para ello hace uso de la función  .codificar().

In [276]:
def mostrar_tablero(tab):
    fila = ""
    for i in range(len(tab)):
        for j in range(len(tab[i])):
            fila = fila + codificar(tab[i][j]) + "\t"
        print(fila)
        fila = ""

**Funcion `comp_tablero()` :** Esta funcion recibe como parametros a `tab_actual` y `tab_meta` que son respectivamente el tablero actual y el tablero meta. Por conveniencia se define al tablero meta como aquel que tiene 1 ó -1 en el lado extremo opuesto de donde el jugador inició la partida y 0 en el resto de posiciones.
Se realizan dos bucles. En el primero se efectúa una operación ***and*** (multiplicación lógica) y en el segundo se compara el tablero obtenido con el tablero meta. 


In [277]:
def comp_tablero(tab_actual, tab_meta):
    tab = cp.deepcopy(tab_actual)
    for i in range(len(tab_actual)):
        for j in range(len(tab_actual[i])):
            tab[i][j] = tab_actual[i][j] * abs(tab_meta[i][j])

    for i in range(len(tab_actual)):
        for j in range(len(tab_actual[i])):
            if tab[i][j] != tab_meta[i][j]:
                return False
    return True

**Funcion `fin_juego()`:** Esta función retorna verdadero si alguno de los competidores llega al extremo opuesto, y falso para cualquier otro caso. Dentro de esta función se hace uso de `comp_tablero()`.

In [278]:
def fin_juego():
    if comp_tablero(T, T_Meta_M):
        print("    GANADOR: Maquina")
        return True
    elif comp_tablero(T, T_Meta_H):
        print("    GANADOR: Humano")
        return True
    else:
        return False

### Parte Maquina
**Funcion `distancia()`:** Esta función se encarga de recibir dos coordenadas `p1` y `p2` como parametros y calcula la distancia entre ambas coordenadas, utilizando la fórmula matemática de distancia entre 2 puntos.

In [279]:
def distancia(p1, p2):
    abcisa1, ordenada1 = p1[0], p1[1]
    abcisa2, ordenada2 = p2[0], p2[1]
    return mt.sqrt((abcisa1 - abcisa2) ** 2 + (ordenada1 - ordenada2) ** 2)


**Funcion `valor_h()`:** Esta función devuelve el valor de h(n). En el presente trabajo se ha definido h(n) como la distancia entre la posición promedio actual y la posición promedio meta.

In [280]:
def valor_h(pos_fichas):
    pos_prom_actual = pos_promedio(pos_fichas)
    return distancia(pos_prom_actual, pos_prom_Meta)




**Clase `Estado` :** Esta clase representa un estado cualquiera del juego y posee 2 atributos `tablero` y `pos_fichas` que son respectivamente el tablero y las posiciones de las fichas que definen a ese estado cualquiera.
Dentro de esta clase, se consideraron los siguientes métodos:

| METODO | DESCRIPCION|
|:----------|:----------|
| `__init__`| Sirve para inicializar una instancia de la clase|
| `actualizar_pos()`| Permite actualizar algun elemento de `pos_fichas`. Recibe como parametros a `i` que presenta el id de la ficha a mover, `dx` que representa el desplazamiento en el eje horizontal y `dy` que representa el desplazamiento en el eje vertical.|
| `bus_indice()`| Permite encontrar el índice de algun elemento de `pos_fichas`. Este índice representa el id de la ficha. La función recibe como parametros a `x` y `y` que conjuntamente (x,y) representan el elemento buscado en la lista de posiciones.|
| `actualizar_tablero()`| Permite actualizar el atributo `tablero` de la clase Estado. Recibe como parametros a `i` que representa el id de la ficha, cuya posición se debe actualizar en el tablero, y `valor` que representa el nuevo valor a poner en el tablero.|
| `posicion_ficha()`| Permite obtener la posicionde una ficha específica. Recibe como parametros a `i` que representa el id de la ficha.|
| `__eq__`| Sirve para comparar dos instancias de la misma clase.|

In [281]:
class Estado:
    tablero = []
    pos_fichas = []

    def __init__(self, tablero, pos_fichas):
        self.tablero = tablero
        self.pos_fichas = pos_fichas

    def actualizar_pos(self, i, xn, yn):
        self.pos_fichas[i][0] = self.pos_fichas[i][0] + xn
        self.pos_fichas[i][1] = self.pos_fichas[i][1] + yn

    def bus_indice(self, x, y):
        for i in range(len(self.pos_fichas)):
            for j in range(len(self.pos_fichas)):
                if self.pos_fichas[i][0] == x and self.pos_fichas[i][1] == y:
                    return i

        return -1

    def actualizar_tablero(self, i, valor):
        x = self.pos_fichas[i][0]
        y = self.pos_fichas[i][1]
        self.tablero[x][y] = valor

    def posicion_ficha(self, i):
        xi = self.pos_fichas[i][0]
        yi = self.pos_fichas[i][1]
        return xi, yi

    def __repr__(self):
        return "\n\t" + str(self.tablero)

    def __eq__(self, otro_estado):
        return comp_tablero(self.tablero, otro_estado.tablero)



**Clase `EstadoComp`:** Esta clase representa un estado cualquiera del juego conjuntamente con sus medidas descriptivas. Posee 4 atributos `estado` y`g` y `h` y `ult_ficha_mov` que son respectivamente el estado, el valor g(n), el valor h(n) y el id de la última ficha movida.

Dentro de esta clase, se consideraron los siguientes métodos:

| METODO | DESCRIPCION|
|:----------|:----------|
| `__init__`| Sirve para inicializar una instancia de la clase|
| `feval()`| Para calcular el valor de la función de evaluación, que matemáticamente se expresa como f(n) = g(n)+h(n)|

In [282]:
class EstadoComp:
    estado = None
    g = 0
    h = 0.0
    ult_ficha_mov = 0

    def __init__(self, estado, g, h, ult_ficha_mov):
        self.estado = estado
        self.g = g
        self.h = h
        self.ult_ficha_mov = ult_ficha_mov

    def feval(self):
        return self.g + self.h
    

**Funcion `copiar_estado()`:** Esta función realiza una copia en profundidad utilizando el método `deepcopy()`. De esa manera, construye un nuevo objeto compuesto y luego, recursivamente, inserta copias en él de los objetos encontrados en el original.


In [283]:
def copiar_estado(e):
    return Estado(cp.deepcopy(e.tablero), cp.deepcopy(e.pos_fichas))


**Funcion `reglas_mover()`**: En esta función se identifican todos los movimientos posibles para una posicion especfica. Se generan las nuevas posiciones para cada uno de los 8 movimientos posibles que están en el diccionario `movs`. Posteriormente se valida que la nueva posición esté dentro del tablero y que no esté ocupada. Si se cumple ello, la posicion se inserta en la lista `movs_posibles`.


In [284]:
def reglas_mover(tab, x, y):
    movs_posibles = []

    for id_mov in movs:
        dx, dy = movs[id_mov]
        x_act = x + dx
        y_act = y + dy
        if 0 <= x_act < 6 and 0 <= y_act < 6 and tab[x_act][y_act] == 0:
            movs_posibles.append(id_mov)

    return movs_posibles

**Funcion `reglas_saltar()`:** En esta función se identifican todos los saltos posibles para una posicion especfica. Se generan las nuevas posiciones considerando las 8 direcciones posibles que están en el diccionario `movs`. En este caso se multiplica por 2 dado que se va a saltear una posición. Posteriormente se validan 3 aspectos: la nueva posición debe estar dentro del tablero, la posicion que saltó debe contener una ficha y la nueva posición no debe estar ocupada. Si se cumple ello, la posicion se inserta en la lista `movs_posibles`


In [285]:
def reglas_saltar(tab, x, y):
    movs_posibles = []

    for id_mov in movs:
        dx, dy = movs[id_mov]
        x_act = x + 2 * dx
        y_act = y + 2 * dy

        if 0 <= x_act < 6 and 0 <= y_act < 6 and abs(tab[x + dx][y + dy]) == 1 and tab[x_act][y_act] == 0:
            movs_posibles.append(id_mov)

    return movs_posibles

**Funcion `mover()`:** Esta función recibe 3 parametros, `id_mov` que identifica el movimiento a realizar, `estado` que representa el estado actual y `pos_ficha` que identifica la posición de la ficha que se debe mover. Con todos estos datos, la función genera un nuevo estado y actualiza tanto el tablero como la lista de las posiciones de la ficha.

In [286]:

def mover(id_mov, estado, pos_ficha):
    e = copiar_estado(estado)
    dx, dy = movs[id_mov]

    e.actualizar_tablero(pos_ficha, 0)
    e.actualizar_pos(pos_ficha, dx, dy)
    e.actualizar_tablero(pos_ficha, 1)
    return e

**Funcion `saltar()`:** Esta función recibe 3 parametros: `id_mov` que identifica el tipo de salto a realizar, `estado` que representa el estado actual y `pos_ficha` que identifica la posición de la ficha que debe saltar. Con todos estos datos, la función genera un nuevo estado y actualiza tanto el tablero como la lista de las posiciones de la ficha.

In [287]:
def saltar(id_mov, estado, pos_ficha):
    e = copiar_estado(estado)
    dx, dy = movs[id_mov]

    e.actualizar_tablero(pos_ficha, 0)

    e.actualizar_pos(pos_ficha, 2 * dx, 2 * dy)

    e.actualizar_tablero(pos_ficha, 1)
    return e

**Funcion `jugadas_mover()` :** Esta función recibe como parámetros a `estado` y `id_ficha` que son el estado actual y la ficha a mover respectivamente.  En este apartado se utiliza tanto la funcion `reglas_mover` y la funcion `mover`, de manera que se retorna una lista con todos los estados posibles que resultan de aplicar las reglas de mover. 

In [288]:
def jugadas_mover(estado, id_ficha):
    jugadas = []
    p = estado.pos_fichas
    reglas = reglas_mover(estado.tablero, p[id_ficha][0], p[id_ficha][1])

    for i in range(len(reglas)):
        jugadas.append(mover(reglas[i], estado, id_ficha))

    return jugadas

**Funcion `bucle_saltar()` :** Esta función recursiva es utilizada en `jugadas_saltar()`. Permite llenar la lista `jugadas` con todos los estados resultantes de hacer saltos simples y saltos sucesivos (de ser posible). En este apartado se utiliza tanto la funcion `reglas_saltar` y la funcion `saltar`, de manera que al finalizar se obtiene una lista llena con todos los estados posibles que resultan de aplicar las reglas de saltar. 

In [289]:
def bucle_saltar(jugadas, e, id_ficha):
    p = e.pos_fichas

    movs_posibles = reglas_saltar(e.tablero, p[id_ficha][0], p[id_ficha][1])

    if len(movs_posibles) != 0:
        for i in range(len(movs_posibles)):

            nuevo_e = saltar(movs_posibles[i], e, id_ficha)

            if not pos_repetida(jugadas, nuevo_e, id_ficha):
                jugadas.append(nuevo_e)
                bucle_saltar(jugadas, nuevo_e, id_ficha)



**Funcion `pos_repetida()` :** Esta función permite identificar si un estado generado `nuevo_e` (que está asociado a una ficha especffica con id `k`)  ya está incluido en la lista `jugadas`. Para ello, recorre la lista y va comparando, uno a uno, la posicion de la ficha en los estados ya insertados. Este apartado permite que la máquina no salte infinitamente de una posición a otra.


In [290]:

def pos_repetida(jugadas, nuevo_e, k):
    for i in range(len(jugadas)):
        xi, yi = jugadas[i].posicion_ficha(k)
        x, y = nuevo_e.posicion_ficha(k)
        if x == xi and y == yi:
            return True
    return False

**Funcion `jugadas_saltar()` :** Esta función recibe como parámetros a `estado` y `id_ficha` que son el estado actual y la ficha a mover respectivamente. Asimismo, retorna una lista con todas las jugadas “saltar” posibles. Para ello hace uso de `bucle saltar`.


In [291]:
def jugadas_saltar(estado, id_ficha):
    jugadas = [estado]
    bucle_saltar(jugadas, estado, id_ficha)
    jugadas.pop(0)
    return jugadas

**Funcion `jugadas_posibles()` :** Esta función recibe como parámetro un  estado compuesto `ec`, el cual se utiliza para generar tanto los sucesores que surgen a partir de la función `jugadas_mover` como de los que surgen de la función `jugadas_saltar`. Esta función retorna una lista que contiene todos los estados sucesores posibles para un estado particular.


In [292]:
def jugadas_posibles(ec):
    sucesores = []
    for k in range(n_fichas):
        estados_mover = jugadas_mover(ec.estado, k)
        for i in range(len(estados_mover)):
            nuevo_ec = EstadoComp(estados_mover[i], ec.g + 1, valor_h(estados_mover[i].pos_fichas), k)
            sucesores.append(nuevo_ec)

        estados_saltar = jugadas_saltar(ec.estado, k)
        for i in range(len(estados_saltar)):
            nuevo_ec = EstadoComp(estados_saltar[i], ec.g + 1, valor_h(estados_saltar[i].pos_fichas), k)
            sucesores.append(nuevo_ec)

    return sucesores

**Funcion `seleccion_primero_el_mejor()` :** Esta función utiliza la función `jugadas_posibles()` para generar las jugadas o estados sucesores; además de ello, hace uso de la función `sorted()` para hacer el ordenamiento. En este caso se le especifica a dicha función que, para ordenar, tome como criterio el valor f de cada estado. Al finalizar se retorna el estado con menor valor f.


In [293]:
def seleccion_primero_el_mejor(ec):
    sucesores = jugadas_posibles(ec)
    sucesores_ordenados = sorted(sucesores, key=usa_f)

    return sucesores_ordenados[0]

**Funcion `usa_f()` :** Esta función encapsula el valor a utilizar para el ordenamiento. En este caso el valor obtenido al calcular la función de evaluación en el estado n.

In [294]:
def usa_f(ec):
    return ec.feval()

**Funcion `estrategia_primero_el_mejor()` :** Esta función transforma/traduce las variables globales del programa en un estado `ec`, luego de ello calcula el nuevo estado utilizando la estrategia de Primero el Mejor. Finalmente, actualiza el valor de las variables globales con los valores correspondientes.


In [295]:
def estrategia_primero_el_mejor():
    global T, pos, n_movs_M, ficha_mov
    ec = EstadoComp(Estado(T, pos), n_movs_M, valor_h(pos), ficha_mov)
    nuevo_ec = seleccion_primero_el_mejor(ec)
    T = nuevo_ec.estado.tablero
    pos = nuevo_ec.estado.pos_fichas
    ficha_mov = nuevo_ec.ult_ficha_mov
    n_movs_M += 1

**Funcion `jugar_maquina()` :** Esta función ejecuta la jugada de la máquina. Para ello hace uso de la estrategia Primero el mejor, la cual permite modificar las variables globales: el tablero `T` , la lista de posiciones de la ficha de la máquiana `pos` y otras dos variables más. Luego de que esa acción es ejecutada, evalúa si ha ganado, en caso de que no, se le da pase al oponente humano.

In [296]:
def jugar_maquina():
    print("Ahora le toca jugar a la PC")
    estrategia_primero_el_mejor()
    mostrar_tablero(T)
    if fin_juego():
        inicio()
    else:
        jugar_humano(T)

### PARTE HUMANO

**Funcion `fuera_tablero()` :** En esta función se toma como parámetros las coordenadas de la posición de la ficha escogida, así como el tablero actual. La función retorna **True** si la posición elegida se encuentra dentro del tablero y **False**, si no.

In [297]:
def fuera_tablero(x, y, tab):
    return not (0 <= x < len(tab) and 0 <= y < len(tab))



**Funcion `act_pos()` :** En la función se toma como parámetro tanto la posición actual de la ficha y la posición en donde se ha de mover, así como el tablero actual del juego. La función cambia el valor de la posición actual a 0 y la posición final a -1 en el tablero.

In [298]:
def act_pos(x, y, x1, y1, tab):
    for i in range(len(tab)):
        if x == pos[i][0] and y == pos[i][1]:
            pos[i][0] = x1
            pos[i][1] = y1

    tab[x][y] = 0
    tab[x1][y1] = -1


**Funcion `direccion_mover()` :** En esta función se toma como parámetro la posición de la ficha a mover, así como el índice del movimiento que se realiza. Esta función retorna la posición final a donde se movería la ficha dependiendo de la dirección tomada.

In [299]:
def direccion_mover(x, y, id_mov):
    tx = ty = 0

    if id_mov in movs:
        dx, dy = movs[id_mov]
        tx = x + dx
        ty = y + dy

    return tx, ty

**Funcion `direccion_saltar()` :** En esta función se toma como parámetro la posición de la ficha a mover, así como el índice del movimiento que se realiza. Esta función retorna la posición final a donde saltaría la ficha dependiendo de la dirección tomada.

In [300]:
def direccion_saltar(x, y, id_mov):
    tx = ty = 0

    if id_mov in movs:
        dx, dy = movs[id_mov]
        tx = x + 2 * dx
        ty = y + 2 * dy

    return tx, ty

**Funcion `comprobar()` :** Esta función toma como parámetros una posición, y el tablero actual. Lo que realiza la función es verificar que a partir de la posición brindada se puede seguir saltando hacia otra dirección permitida. Si se permite el salto se almacena en el diccionario `comp`, el cual será el retorno de la función.

In [301]:
def comprobar(x, y, tab, saltos_realizados):
    comp = {}

    for i in movs:
        dx, dy = movs[i]
        tx = x + dx
        ty = y + dy

        if not fuera_tablero(tx, ty, tab) and abs(tab[tx][ty]) == 1:
            tx = x + 2*dx
            ty = y + 2*dy

            if not fuera_tablero(tx, ty, tab) and tab[tx][ty] == 0:
                if (tx, ty) not in saltos_realizados:
                    comp[i] = (tx, ty)

    return comp

**Funcion `continuar()` :** Esta función recibe como parámetros la posición de la ficha en cuestión, el tablero actual y `visitados` el cual es un set que almacena las posiciones a las que puede saltar la ficha.  En esta función se valida a partir de `comprobar()` las posiciones a donde se pueda seguir saltando, dado que la función es recursiva se debe asegurar de no recorrer las posiciones que ya hayan sido almacenadas.

In [302]:
def continuar(x, y, tab, permitidos, visitados):
    jugada = comprobar(x, y, tab, visitados)

    if jugada:
        for i in jugada:
            visitados.append(jugada[i])


    if jugada:
        for i in jugada:
            x1, y1 = jugada[i]
            act_pos(x, y, x1, y1, tab)
            permitidos.append((x1, y1))
            continuar(x1, y1, tab, permitidos, visitados)

    return permitidos

**Funcion `mover_ficha()` :** Esta función toma como parámetro la posición elegida por el usuario y el tablero actual. Esta función es la que determina todos los movimientos posibles para la ficha; primero verificando si la nueva posición está fuera del tablero; si ese no es el caso, se comprueba si es de salto y/o si puede seguir saltando, almacenando las posiciones si es el caso. Dejando como alternativa, que la posición es de movimiento simple. Todos las posiciones finales están en el set `jug` el cual es retornado por la función.

In [303]:
def mover_ficha(x, y, tab):
    print(f"\nLa ficha {x, y} se puede mover a:")
    jug = []
    tablero = cp.deepcopy(tab)

    for i in movs:
        x1, y1 = direccion_mover(x, y, i)

        if fuera_tablero(x1, y1, tablero):
            continue

        elif abs(tablero[x1][y1]) == 1:

            x1, y1 = direccion_saltar(x, y, i)

            if fuera_tablero(x1, y1, tablero):
                continue

            elif abs(tablero[x1][y1]) == 1:
                continue

            else:
                jug += [(x1, y1)]
                act_pos(x, y, x1, y1, tablero)
                jug += continuar(x1, y1, tablero, [], [(x, y)])

        else:
            jug += [(x1, y1)]
            act_pos(x, y, x1, y1, tablero)

    return jug

**Funcion `validar_movimiento()` :** Esta función se encarga de recibir la ficha elegida por el jugador y verifica si no es una posición vacía o una ficha de la pc, así como si se encuentra en el tablero.

In [304]:
def validar_movimiento(tab):
    x0, y0 = input("\nQue ficha desea mover: ").split(",")
    x0 = int(x0)
    y0 = int(y0)

    if fuera_tablero(x0, y0, tab):
        print("\nUbicación fuera del tablero")
        x0,y0 = validar_movimiento(tab)

    elif tab[x0][y0] == 0:
        print("\nEn esta ubicación no hay ninguna ficha")
        x0,y0 = validar_movimiento(tab)

    elif tab[x0][y0] == 1:
        print("\nEsta ficha pertenece a la PC")
        x0,y0 = validar_movimiento(tab)


    return x0, y0

**Funcion `jugar_humano()` :** Esta función toma como parámetro el tablero actual del juego, primero registra la posición elegida por el usuario y a través de jugadas muestra los posibles movimientos que puede realizar. Para posteriormente preguntarle cual de todos ellos escoge y actualizar el tablero según lo elegido. Dando por último paso a `jugar_maquina()`.

In [305]:
def jugar_humano(tab):

    x0, y0 = validar_movimiento(tab)

    jugadas = mover_ficha(x0, y0, tab)

    for i, coords in enumerate(jugadas):
        print(f"{i + 1}. {coords}")

    opc = int(input("Que ubicacion elige: "))

    act_pos(x0, y0, jugadas[opc - 1][0], jugadas[opc - 1][1], tab)

    mostrar_tablero(tab)
    if fin_juego():
        inicio()
    else:
        jugar_maquina()

**Funcion `inicio()` :** Esta función llama a `estado_inicial()` y muestra el menú de opciones. Dependiendo de la opción seleccionada, se le da paso a la máquina o al humano para que realicen su jugada; o también se finaliza el juego en casi sea especificado.

In [306]:
def inicio():
    estado_inicial()
    print("""\n
          ------ ESQUINAS CHINAS -----
          Le gustaria comenzar el juego?
          [1] Si
          [2] NO
          [3] Salir
    """)
    
    opcion = int(input("Seleccione la opcion: \n"))
    mostrar_tablero(T)
    
    if opcion == 1:
        print('Oponente inicia el juego')
        jugar_humano(T)
        
    elif opcion == 2:
        jugar_maquina()
        
    elif opcion == 3:
        print("\n")
        print("Saliendo ...")
        

**Parte `__main__` :** En esta parte de código se ejecuta todo el juego.

In [307]:
if __name__ == '__main__':
    inicio()



          ------ ESQUINAS CHINAS -----
          Le gustaria comenzar el juego?
          [1] Si
          [2] NO
          [3] Salir
    
Seleccione la opcion: 
3
X	X	X	-	-	-	
X	X	-	-	-	-	
X	-	-	-	-	-	
-	-	-	-	-	O	
-	-	-	-	O	O	
-	-	-	O	O	O	


Saliendo ...


## DESARROLLO DE LA INTERFAZ
### Parte Final

**Instalación de la libreria pygame:** Es importante que antes de compilar las demas lineas de código se halla descargardo la librería *pygame*. Debido a que es la libería base que permitirá la visualización del juego inteligente.

In [ ]:
!pip install pygame

**Importación de Bibliotecas:** Para la codificación del juego inteligente mediante una interfaz se utilizaron seis bibliotecas: *pygame* para las funciones de visualizacion, *sys* para la utilización del método `sys.exit()`, time para el control del tiempo de la partida, os para la lectura de los archivos, random para la realización de la serpentina en el caso ganador y numpy para el uso de arreglos.

In [3]:
import pygame
import sys
import time
import os
import math
import random
import numpy as np

pygame 2.4.0 (SDL 2.26.4, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


**Variables Globales:** Entre las variables globales que se definieron, se tiene:
- `NUM_FILAS`  → Número de Filas del Tablero.

- `NUM_COLUMNAS` → Número de Columnas del Tablero.

- `TAM_CASILLA` → Tamaño de las casillas.

- `TAM_TABLERO` → Tamaño del tablero.

- `COLORES UTILIZADOS`	→ Colores RGB utilizados.

- `FICHA_GANADORA` → Representa el jugador ganador (M/H).

- `IMAGENES` → Imagenes importadas.

- `x0` → Fila seleccionada en el tablero.

- `y0` → Columna seleccionada en el tablero.

- `jugadas` → Array con las posibles jugadas de las fichas.

- `confetti_list` → Contiene varias clases confetis para ser visualizadas.

- `BAL` → Array que mide las posicones cuando se le da clic

In [4]:
#Dimensiones del Tablero
NUM_FILAS = 6
NUM_COLUMNAS = 6
TAM_CASILLA = 95
TAM_TABLERO = (14) * (58)

#Colores Utilizados
BLANCO = (255, 255, 255)
GRIS = (219,219,219)
TURQ = (234,255,239)
NEGRO = (0,0,0)
F_SOM = (20,101,209)
C_F1 = (0, 74, 173)   # Color Ficha 1
C_F2 = (12, 192, 223) # Color Ficha 2
C_RES = (187,233,241)
FONDO=(234,255,239)

#Imagenes
imagen_victoria = pygame.image.load('Trofeo.png')
imagen_derrota = pygame.image.load('Derrota.png')

#Adiconales
FICHA_GANADORA = ''
x0 =-1
y0 = -1
jugadas = []
confetti_list = []
BAL = []

**Funcion `Comp_Meta()` :** Esta función toma como parámetros el `tab_actual`, `tab_meta` y `ficha`. Siendo estos, el tablero actual, el tablero meta  la ficha del jugador (esta puede ser -1 o 1 dependiendo de si se trata del humano o la máquina). En el `for` se realiza el conteo de las similitudes dentro de los dos tableros previamente mencionados. Finalmente todo se almacena en `con`, el cual es el valor que es retornado.

In [310]:
def Comp_Meta(tab_actual, tab_meta,ficha):
    cont =0
    for i in range(len(tab_actual)):
        for j in range(len(tab_actual[i])):
            if(tab_actual[i][j]==tab_meta[i][j]==ficha):
                cont += 1
    return cont

**Funcion `fin_juego()` :** La funciona invoca a `FICHA_GANADORA`, luego invoca a la funcion `comp_tablero()` para poder conocer si la ficha ganadora es de la máquina o del humano. Finalmente el valor se almacena dentro de `FICHA_GANADORA`.

In [311]:
def fin_juego():
    global FICHA_GANADORA
    if comp_tablero(T, T_Meta_M):
        FICHA_GANADORA = 'M'
    elif comp_tablero(T, T_Meta_H):
        FICHA_GANADORA = 'H'

**Funcion `jugar_maquina()` :** La funciona invoca a las funciones `estrategia_primero_el_mejor()` y `fin_juego()`.

In [312]:
def jugar_maquina():
    estrategia_primero_el_mejor()
    fin_juego()

**Funcion `act_pos()` :** La función toma como parámetros a `x`, `y`, `x1`, `y1` y `tab`. Recorre el arreglo y localiza la ficha a mover, en esta intercambia la posicones y se actualiza la posicion inicial.

In [313]:
def act_pos(x, y, x1, y1, tab):
    for i in range(len(tab)):
        if x == pos[i][0] and y == pos[i][1]:
            pos[i][0] = x1
            pos[i][1] = y1

    tab[x][y] = 0
    tab[x1][y1] = -1

**Funcion `mover_ficha()` :** La función toma como parámetros a `x`, `y`y `tab`. Lo que realiza esta funcion es la identificacion de posibles jugadas, esto es posibles gracias a las funciones `direccion_saltar()`y `direccion_mover`. Finalmente una vez identificados los movimientos se retorna el arreglo con los movimientos.

In [314]:
def mover_ficha(x, y, tab):
    jug = []
    tablero = cp.deepcopy(tab)

    for i in movs:
        x1, y1 = direccion_mover(x, y, i)

        if fuera_tablero(x1, y1, tablero):
            continue

        elif abs(tablero[x1][y1]) == 1:

            x1, y1 = direccion_saltar(x, y, i)

            if fuera_tablero(x1, y1, tablero):
                continue

            elif abs(tablero[x1][y1]) == 1:
                continue

            else:
                jug += [(x1, y1)]
                act_pos(x, y, x1, y1, tablero)
                jug += continuar(x1, y1, tablero, [], [(x, y)])

        else:
            jug += [(x1, y1)]
            act_pos(x, y, x1, y1, tablero)

    return jug

**Función `GetPath()`:** Esta función nos devolverá una ruta, la cual usaremos como la absoluta para luego poder llamar archivos que tenemos dentro de la carpeta que estaremos usando cuando se requiera el uso de una imagen, canción o tipo de fuente.

In [315]:
def GetPath(Archivo):
    ruta_absoluta = os.path.join(os.getcwd(), Archivo)
    return ruta_absoluta

In [316]:
OpenSans = GetPath("OpenSans.ttf")
fonttt = GetPath("font.ttf")

**Función `Botones()`:** Dentro de esta función tendremos los botones de `BotonPartida` y `BotonJuego`, con los cuales al presionarlos la primera nos abrirá la nueva interfaz en donde se mostrará el tablero del juego y el segundo para cerrar la aplicacion.

In [317]:
def Botones():
    BotonPartida.dibujar(ventana_2)
    BotonJuego.dibujar(ventana_2)
    mouse_pos = pygame.mouse.get_pos()
    if BotonPartida.detectar_clic(mouse_pos):
        Posciones.append(mouse_pos)
    elif BotonJuego.detectar_clic(mouse_pos):
        Posciones.append(mouse_pos)

**Función `Imprimir()`:** Esta función recibe como parámetros el titulo, los valores de la esclaa y la posicones dentro del tablero. Esto se realiza mediante un renderizado y un escalado a través de `convert_alpha()` y `transform.smoothscale()`.

In [318]:
def Imprimir(titulo, t1,t2,x,y):
    a = texto.render(titulo,False, NEGRO)
    a = a.convert_alpha()
    a = pygame.transform.smoothscale(a, (t1, t2))
    ventana_2.blit(a,(x, y))

**Función `ImprimirNegrita()`:** Esta función recibe como parámetros el titulo, los valores de la esclaa y la posicones dentro del tablero. Esto se realiza mediante un renderizado y un escalado a través de `convert_alpha()` y `transform.smoothscale()`. A diferencia de la función anterior saldrá el texto en formato negrita.

In [319]:
def ImprimirNegrita(titulo, t1,t2,x,y):
    a = Meta.render(titulo,False, NEGRO)
    a = a.convert_alpha()
    a = pygame.transform.smoothscale(a, (t1, t2))
    ventana_2.blit(a,(x, y))

**Función `Botonesv2()`:** Dentro de esta otra función tendremos los botones pertenecientes a la primera interfaz, el primero y el último que son de ***Jugar*** y de ***Salir***, de los cuales como se indica, son para iniciar el juego y para salir de este respectivamente, y también otrosa dos botones intermedios, con los cuales el jugador podrá escoger el color de su ficha, ya sea el celeste o el azul.

In [320]:
def Botonesv2(event):
    global C_F1,C_F2,ventana_2,start_time,background,FICHA_GANADORA,jugadas,texto
    MENU_MOUSE_POS = pygame.mouse.get_pos()
    for boton in [Boton1,Boton2,Boton3,Boton4]:
        boton.changeColor(MENU_MOUSE_POS)
        boton.update(ventana_1)
    if event.type == pygame.MOUSEBUTTONDOWN:
        if Boton1.checkForInput(MENU_MOUSE_POS):
            pygame.display.quit()
            ventana_2 = pygame.display.set_mode((TAM_TABLERO, TAM_TABLERO))
            background = pygame.image.load(GetPath("TableroPor.png"))
            pygame.display.set_icon(icono)
            pygame.display.set_caption("Tablero de Esquinas Chinas")
            estado_inicial()
            jugadas=[]
            texto=pygame.font.Font(OpenSans, 150)
            FICHA_GANADORA = ''

            # Tiempo inicial
            start_time = time.time()
        if Boton2.checkForInput(MENU_MOUSE_POS):
            C_F1 = (0, 74, 173)   # Color Ficha 1
            C_F2 = (12, 192, 223) # Color Ficha 2
        if Boton3.checkForInput(MENU_MOUSE_POS):
            C_F1 = (12, 192, 223)   # Color Ficha 1
            C_F2 = (0, 74, 173)     # Color Ficha 2
        if Boton4.checkForInput(MENU_MOUSE_POS):
            pygame.quit()
            sys.exit()

**Función `Contador()`:** Dentro de esta función se utilizará la librería **time**, esta servirá para pode medir el tiempo transcurrido a través de la variable `elapsed_time`. Asimismo, Estos tiempo se colocarán en la variable `time_strs` para que pueda ser reconocido como formato texto. Por ultimo será mostrado dentro de la ventana.

In [ ]:
def Contador():
    # Tiempo transcurrido
    elapsed_time = time.time() - start_time
    # Cálculo de las horas, minutos y segundos
    hours = int(elapsed_time / 3600)
    minutes = int((elapsed_time - (hours * 3600)) / 60)
    seconds = int(elapsed_time - (hours * 3600) - (minutes * 60))

    time_str = "{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds)

    time_surface = font.render(time_str, False, NEGRO)
    time_surface = time_surface.convert_alpha()
    scaled_surface = pygame.transform.smoothscale(time_surface, (95, 25))
    scaled_surface
    ventana_2.blit(scaled_surface, (98, 67))

**Función `Jugadores()`:** Esta función utiliza las funciones `Imprimir()` e `ImprimirNegrita()` y `Comp_Meta()` (Utilizado como parámetro para las dos anteriores).

In [321]:
def Jugadores():
    Imprimir("Jugador Computadora", 250,30,450,48)
    ImprimirNegrita("Meta: "+ str(Comp_Meta(T, T_Meta_M,1)), 70,25,540,85)
    Imprimir("Jugador Persona", 180,30,120,710)
    ImprimirNegrita("Meta: "+ str(Comp_Meta(T, T_Meta_H,-1)), 70,25,170,750)

**Función `Tablero()`:** Esta función se crea con el fin de que se muestre en la segunda interfaz el tablero, con los bordes y los respectivos colores de cada cuadro.

In [322]:
def Tablero(fila, columna, evento):
    global Posciones, jugadas
    init=(fila,columna)
    if(init in jugadas and len(BAL)!=0):
        cuadrado = pygame.draw.rect(ventana_2, C_RES , pygame.Rect(columna * TAM_CASILLA+120,fila * TAM_CASILLA+120, TAM_CASILLA, TAM_CASILLA))
        mouse_pos = pygame.mouse.get_pos()
        if cuadrado.collidepoint(mouse_pos):
            Posciones.append(mouse_pos)
            if evento.type == pygame.MOUSEBUTTONDOWN:
                act_pos(x0, y0, fila, columna, T)
                jugadas=[]
                time.sleep(1)
                jugar_maquina()
    else:
        if((fila+columna)%2==0):
            pygame.draw.rect(ventana_2, BLANCO , pygame.Rect(columna * TAM_CASILLA+120,fila * TAM_CASILLA+120, TAM_CASILLA, TAM_CASILLA))
        else:
            pygame.draw.rect(ventana_2, GRIS , pygame.Rect(columna * TAM_CASILLA+120,fila * TAM_CASILLA+120, TAM_CASILLA, TAM_CASILLA))
    pygame.draw.rect(ventana_2, NEGRO , pygame.Rect(columna * TAM_CASILLA+120,fila * TAM_CASILLA+120, TAM_CASILLA, TAM_CASILLA),1)

**Función `Fichas()`:** Con esta función se muestran las fichas, tanto de la computadora como del jugador. También detecta si el mouse se encuentra encima de alguna de las fichas del jugador y así cambiar el color de la ficha a un color negro.

In [323]:
def Fichas(fila, columna):
    global BAL, jugadas, Posciones,x0,y0
    init=T[fila][columna]
    if(init == 1):
        pygame.draw.circle(ventana_2, C_F2, (columna * TAM_CASILLA + TAM_CASILLA+ 145 // 2, fila * TAM_CASILLA + TAM_CASILLA+ 148 // 2), 34)
        
    elif(init == -1):
        circle_pos=[columna * TAM_CASILLA + TAM_CASILLA+ 145 // 2,fila * TAM_CASILLA + TAM_CASILLA+ 148 // 2]
        
        # Detectar si el mouse está encima del círculo
        mouse_pos = pygame.mouse.get_pos()
        distance = math.sqrt((mouse_pos[0] - circle_pos[0])**2 + (mouse_pos[1] - circle_pos[1])**2)
        
        if distance <= 34:
            # El mouse está encima del círculo, cambiar el color
            pygame.draw.circle(ventana_2, F_SOM, (circle_pos[0], circle_pos[1]), 34)

            Posciones.append(mouse_pos)
            if evento.type == pygame.MOUSEBUTTONDOWN:
                pygame.time.delay(100) # Espera 100 milisegundos (0.1 segundos)

                if len(BAL) == 0 :
                    jugadas = mover_ficha(fila, columna, T)
                    x0=fila
                    y0=columna
                    BAL.append(fila)
                else:
                    BAL=[]
        else:
            pygame.draw.circle(ventana_2, C_F1, (circle_pos[0], circle_pos[1]), 34)

**Función `get_font()`:** Esta funcion lo que retorna es el fond que será utilizado para mostrarse. Toma como parámetro `size`, el cual se utiliza dentro de `pygame.font.Font()`.

In [324]:
def get_font(size): # Returns Press-Start-2P in the desired size
    return pygame.font.Font(fonttt, size)

**Clase `Boton` :** Esta clase representa el tipo de botón utilizados para la segunda interfaz, entre las variables utilizadas en el constructor se puede mencionar a `x`, `y`,`w`, `h`,`color` y `texto`. Estos son de utilizada para identificar la dimensión, la poscición y el texto que mantendrá el botón. Dentro de esta clase, se consideraron los siguientes métodos:

| METODO | DESCRIPCION|
|:----------|:----------|
| `__init__`| Sirve para inicializar una instancia de la clase|
| `dibujar()`| Lo que realiza es la visualización de un rectangulo con forma de botón en la ventana cuya variable se encuentra expresada por `superficie`.|
| `detectar_clic()`| Identifca si la posicion que se envia es la misma que la que se encuentra localizado el botón, para ello utiliza el parámtero `pos`, respresentando como la posicion a comparar.|

In [325]:
class Boton:
    def __init__(self, x, y, w, h, color, texto):
        self.rect = pygame.Rect(x, y, w, h)
        self.color = color
        self.texto = texto
        self.font = pygame.font.Font(None, int(h * 0.6))

    def dibujar(self, superficie):
        pygame.draw.rect(superficie, self.color, self.rect, border_radius=4)
        texto = self.font.render(self.texto, True, BLANCO)
        text_rect = texto.get_rect(center=self.rect.center)
        superficie.blit(texto, text_rect)

    def detectar_clic(self, pos):
        return self.rect.collidepoint(pos)

**Clase `BotonV2` :** Esta clase representa una instancia del tipo de botón utilizados en la primera interfaz, consta de los atributos, `image`, `pos`, `text_input`, `font`, `base_color` y `hovering_color`. Estos servirán para la correcta creación y actualización de los botones dentro de la primera interfaz realizada. Dentro de esta clase, se consideraron los siguientes métodos: :

| METODO | DESCRIPCION|
|:----------|:----------|
| `__init__`| Sirve para inicializar una instancia de la clase.|
| `update()`| Lo que realiza es la visualizacion del boton dentro de la expresada por la variable `screen`.|
| `checkForInput()`| Verifica que dado que el parámetro `position`, se encuentre dentro las dimensiones del boton.|
| `ChangeColor()`| Realiza el cambio de colocar en caso se haya determinado que el parámetro `position`, se encuentra dentro de las dimensiones del boton.|

In [326]:
class BotonV2():
    def __init__(self, image, pos, text_input, font, base_color, hovering_color):
        self.image = image
        self.x_pos = pos[0]
        self.y_pos = pos[1]
        self.font = font
        self.base_color, self.hovering_color = base_color, hovering_color
        self.text_input = text_input
        self.text = self.font.render(self.text_input, True, self.base_color)
        if self.image is None:
            self.image = self.text
        self.rect = self.image.get_rect(center=(self.x_pos, self.y_pos))
        self.text_rect = self.text.get_rect(center=(self.x_pos, self.y_pos))

    def update(self, screen):
        if self.image is not None:
            screen.blit(self.image, self.rect)
        screen.blit(self.text, self.text_rect)

    def checkForInput(self, position):
        if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
            return True
        return False

    def changeColor(self, position):
        if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
            self.text = self.font.render(self.text_input, True, self.hovering_color)
        else:
            self.text = self.font.render(self.text_input, True, self.base_color)

**Clase `Confetti` :** Esta clase representa una instancia de los confetis utilizados en la segunda interfaz cuando un jugador termina siendo vencedor, para ello se crean posiciones, velocidades y colores random. Dentro de esta clase, se consideraron los siguientes métodos: :

| METODO | DESCRIPCION|
|:----------|:----------|
| `__init__`| Sirve para inicializar una instancia de la clase.|
| `fall()`| Mide la velocidad en la que caerá la serpentina dentro del tablero.|
| `draw()`| Dibuja un circulo, estos harán la simulacion de la serpentina, su radio variará de manera aleatoria entre 5 a 10.|

In [ ]:
class Confetti:
    def __init__(self):
        self.x = random.randint(0, TAM_TABLERO)
        self.y = random.randint(-TAM_TABLERO, -10)
        self.speed = random.randint(1, 5)
        self.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        self.size = random.randint(5, 10)

    def fall(self):
        self.y += self.speed
        if self.y > TAM_TABLERO:
            self.y = random.randint(-TAM_TABLERO, -10)

    def draw(self):
        pygame.draw.circle(ventana_2, self.color, (self.x, self.y), self.size)

**Variables de Instancia:** Entre las variables que serán instancias se tienen:
- `background`  → El fondo del tablero.

- `icono` → El icono del aplicativo.

- `font-texto-Meta` → El font de un texto con Opensans.

- `BotonPartida` → Boton para comenzar una partida.

- `BotonJuego` → Boton para salir del juego.

In [327]:
pygame.init()

# Musica
pygame.mixer.music.load(GetPath("Cancion_2.mp3"))

# Importar imagen
background = pygame.image.load(GetPath("fondo.png"))

# Cargar el icono
icono = pygame.image.load(GetPath("iconos.png"))

# Establecer el icono de la ventana
pygame.display.set_icon(icono)
pygame.display.set_caption("Tablero de Esquinas Chinas")

# Definir fuentes
font = pygame.font.SysFont(OpenSans, 250)
texto = pygame.font.SysFont(OpenSans, 250)
Meta = pygame.font.SysFont(OpenSans, 500)

# Reproducción en Bucle
pygame.mixer.music.play(-1)

# Botones
BotonPartida = Boton(522, 730, 170, 50, (45,0,153), 'Nueva Partida')
BotonJuego = Boton(330, 730, 170, 50, NEGRO, 'Salir del Juego')

En este apartado se realizan las instancias de los botonos haciendo uso de la clase `BotonV2`

In [328]:
Boton1 = BotonV2(image=pygame.image.load(GetPath("Options Rect.png")), pos=(350,250), 
                            text_input="JUGAR", font=get_font(35), base_color=TURQ, hovering_color="Black")
Boton2 = BotonV2(image=pygame.image.load(GetPath("1.png")), pos=(350, 350), 
                            text_input="   FICHA    ", font=get_font(35), base_color=C_F1, hovering_color="Black")
Boton3 = BotonV2(image=pygame.image.load(GetPath("2.png")), pos=(350, 450), 
                            text_input="   FICHA    ", font=get_font(35), base_color=C_F2, hovering_color="Black")
Boton4  = BotonV2(image=pygame.image.load(GetPath("Options Rect.png")), pos=(350, 550), 
                            text_input="SALIR", font=get_font(35), base_color=GRIS, hovering_color="Black")

Se rellena la cantidad de confetti que aparecerá a través de `confetti_list()`.

In [332]:
for i in range(500):
    confetti_list.append(Confetti())

Se inicializan las ventanas. La primera aparecerá de acuerdo a las dimensiones previamente definidas de la constante `TAM_TABLERO`. Asimismo, la segunda ventana se mantendrá vacía hasta que se realice el juego.

In [333]:
ventana_1 = pygame.display.set_mode((TAM_TABLERO, TAM_TABLERO))
ventana_2 = None

Toda la lógica se identifica en la siguiente codificación. Lo más resaltante es el uso de evento mediante la función `event.get()`. 

**IMPORTANTE**: Para poder visualizar el juego de manera correcta ejecutar la celda con el apartado *CODIGO COMPLETO*

In [331]:
while True:
    # Manejo de eventos
    for evento in pygame.event.get():
        if evento.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    if ventana_2 is None:
        # Limpiar la ventana
        ventana_1.blit(background, (0, 0))
        iniciar=[]

        # Dibujar el tablero
        texto=pygame.font.Font(fonttt, 150)
        b = texto.render("Esquinas Chinas",False, NEGRO)
        b = b.convert_alpha()
        b = pygame.transform.smoothscale(b, (400, 110))
        ventana_1.blit(b,(150, 60))
        #Agregar los botones
        Botonesv2(evento)

        pygame.display.flip()
        pygame.time.delay(10)
    else:
        # Limpiar la ventana
        ventana_2.blit(background, (0, 0))
        iniciar=[]


        Posciones=[]
        Contador()
        Botones()
        Jugadores()

        if(FICHA_GANADORA!='M' and FICHA_GANADORA!='H'):
        # Dibujar el tablero
            for fila in range(NUM_FILAS):
                for columna in range(NUM_COLUMNAS):
                    Tablero(fila, columna,evento)
                    Fichas(fila, columna)

        else:
            if FICHA_GANADORA =='H':
                ventana_2.blit(imagen_victoria, (0,0))
                for confetti in confetti_list:
                    confetti.fall()
                    confetti.draw()
            else:
                ventana_2.blit(imagen_derrota, (0,0))

        mouse_pos = pygame.mouse.get_pos()

        if mouse_pos in Posciones:
            pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_HAND)
        else:
            pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_ARROW)

        if evento.type == pygame.MOUSEBUTTONDOWN:
            if BotonPartida.detectar_clic(pygame.mouse.get_pos()):
                pygame.display.quit()
                ventana_2=None
                ventana_1 = pygame.display.set_mode((TAM_TABLERO, TAM_TABLERO))
                background = pygame.image.load(GetPath("fondo.png"))
                pygame.display.set_icon(icono)
                pygame.display.set_caption("Tablero de Esquinas Chinas")
            if BotonJuego.detectar_clic(pygame.mouse.get_pos()):
                pygame.quit()

        # Actualizar la ventana
        pygame.display.flip()

error: video system not initialized

## Codigo Completo

En este apartado se encuentran todas las clases previamente definidas tanto en la parte **lógica** como en la parte de la **interfaz**. Ejecutar la siguiente celda para visualizar el juego inteligente de manera *gráfica*.

In [ ]:
import pygame
import sys
import time
import os
import math
import random
import numpy as np
import math as mt
import copy as cp

# Definiendo las variables globales
T = []
T_Meta_M = []
T_Meta_H = []
pos = []
pos_Meta_M = []
pos_prom_Meta = []
n_fichas = 6
n_movs_M = 0
ficha_mov = -1
movs = {}

#Dimensiones del Tablero
NUM_FILAS = 6
NUM_COLUMNAS = 6
TAM_CASILLA = 95
TAM_TABLERO = (14) * (58)

#Colores Utilizados
BLANCO = (255, 255, 255)
GRIS = (219,219,219)
TURQ = (234,255,239)
NEGRO = (0,0,0)
F_SOM = (20,101,209)
C_F1 = (0, 74, 173)   # Color Ficha 1
C_F2 = (12, 192, 223) # Color Ficha 2
C_RES = (187,233,241)
FONDO=(234,255,239)

#Imagenes
imagen_victoria = pygame.image.load('Trofeo.png')
imagen_derrota = pygame.image.load('Derrota.png')

#Adiconales
FICHA_GANADORA = ''
x0 =-1
y0 = -1
jugadas = []
confetti_list = []
BAL = []

def pos_promedio(pos_fichas):
    sumx = sumy = 0.0
    for i in range(len(pos_fichas)):
        sumx = sumx + pos_fichas[i][0]
        sumy = sumy + pos_fichas[i][1]
    promx = sumx / n_fichas
    promy = sumy / n_fichas
    prom = [promx, promy]
    return prom


def obtener_pos(tab, jugador):
    p = []
    
    for i in range(len(tab)):
        
        for j in range(len(tab[i])):
            
            if tab[i][j] == jugador:
                posicion = [i, j]
                p.append(posicion)
    return p


def estado_inicial():
    global T, T_Meta_M, T_Meta_H

    T = [[1, 1, 1, 0, 0, 0],
         [1, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0,-1],
         [0, 0, 0, 0,-1,-1],
         [0, 0, 0,-1,-1,-1]
         ]

    T_Meta_M = [[0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 1, 1],
                [0, 0, 0, 1, 1, 1]
                ]

    T_Meta_H = [[-1,-1,-1, 0, 0, 0],
                [-1,-1, 0, 0, 0, 0],
                [-1, 0, 0, 0, 0, 0],
                [ 0, 0, 0, 0, 0, 0],
                [ 0, 0, 0, 0, 0, 0],
                [ 0, 0, 0, 0, 0, 0]
                ]
    global pos, pos_Meta_M
    pos = [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [2, 0]]
    pos_Meta_M = [[3, 5], [4, 4], [4, 5], [5, 3], [5, 4], [5, 5]]

    global movs
    movs = {
        1: (-1, 0),
        2: (1, 0),
        3: (0, -1),
        4: (0, 1),
        5: (-1, -1),
        6: (1, 1),
        7: (-1, 1),
        8: (1, -1)
    }
    global pos_prom_Meta
    pos_prom_Meta = pos_promedio(pos_Meta_M)

def codificar(v):
    if v == -1:
        return 'O'
    elif v == 1:
        return 'X'
    elif v == 0:
        return "-"


def comp_tablero(tab_actual, tab_meta):
    tab = cp.deepcopy(tab_actual)
    for i in range(len(tab_actual)):
        for j in range(len(tab_actual[i])):
            tab[i][j] = tab_actual[i][j] * abs(tab_meta[i][j])

    for i in range(len(tab_actual)):
        for j in range(len(tab_actual[i])):
            if tab[i][j] != tab_meta[i][j]:
                return False
    return True

def distancia(p1, p2):
    abcisa1, ordenada1 = p1[0], p1[1]
    abcisa2, ordenada2 = p2[0], p2[1]
    return mt.sqrt((abcisa1 - abcisa2) ** 2 + (ordenada1 - ordenada2) ** 2)


def valor_h(pos_fichas):
    pos_prom_actual = pos_promedio(pos_fichas)
    return distancia(pos_prom_actual, pos_prom_Meta)


class Estado:
    tablero = []
    pos_fichas = []

    def __init__(self, tablero, pos_fichas):
        self.tablero = tablero
        self.pos_fichas = pos_fichas

    def actualizar_pos(self, i, xn, yn):
        self.pos_fichas[i][0] = self.pos_fichas[i][0] + xn
        self.pos_fichas[i][1] = self.pos_fichas[i][1] + yn

    def bus_indice(self, x, y):
        for i in range(len(self.pos_fichas)):
            for j in range(len(self.pos_fichas)):
                if self.pos_fichas[i][0] == x and self.pos_fichas[i][1] == y:
                    return i

        return -1

    def actualizar_tablero(self, i, valor):
        x = self.pos_fichas[i][0]
        y = self.pos_fichas[i][1]
        self.tablero[x][y] = valor

    def posicion_ficha(self, i):
        xi = self.pos_fichas[i][0]
        yi = self.pos_fichas[i][1]
        return xi, yi

    def __repr__(self):
        return "\n\t" + str(self.tablero)

    def __eq__(self, otro_estado):
        return comp_tablero(self.tablero, otro_estado.tablero)

class EstadoComp:
    estado = None
    g = 0
    h = 0.0
    ult_ficha_mov = 0

    def __init__(self, estado, g, h, ult_ficha_mov):
        self.estado = estado
        self.g = g
        self.h = h
        self.ult_ficha_mov = ult_ficha_mov

    def feval(self):
        return self.g + self.h
    
def copiar_estado(e):
    return Estado(cp.deepcopy(e.tablero), cp.deepcopy(e.pos_fichas))

def reglas_mover(tab, x, y):
    movs_posibles = []

    for id_mov in movs:
        dx, dy = movs[id_mov]
        x_act = x + dx
        y_act = y + dy
        if 0 <= x_act < 6 and 0 <= y_act < 6 and tab[x_act][y_act] == 0:
            movs_posibles.append(id_mov)

    return movs_posibles


def reglas_saltar(tab, x, y):
    movs_posibles = []

    for id_mov in movs:
        dx, dy = movs[id_mov]
        x_act = x + 2 * dx
        y_act = y + 2 * dy

        if 0 <= x_act < 6 and 0 <= y_act < 6 and abs(tab[x + dx][y + dy]) == 1 and tab[x_act][y_act] == 0:
            movs_posibles.append(id_mov)

    return movs_posibles



def mover(id_mov, estado, pos_ficha):
    e = copiar_estado(estado)
    dx, dy = movs[id_mov]

    e.actualizar_tablero(pos_ficha, 0)
    e.actualizar_pos(pos_ficha, dx, dy)
    e.actualizar_tablero(pos_ficha, 1)
    return e


def saltar(id_mov, estado, pos_ficha):
    e = copiar_estado(estado)
    dx, dy = movs[id_mov]

    e.actualizar_tablero(pos_ficha, 0)

    e.actualizar_pos(pos_ficha, 2 * dx, 2 * dy)

    e.actualizar_tablero(pos_ficha, 1)
    return e


def jugadas_mover(estado, id_ficha):
    jugadas = []
    p = estado.pos_fichas
    reglas = reglas_mover(estado.tablero, p[id_ficha][0], p[id_ficha][1])

    for i in range(len(reglas)):
        jugadas.append(mover(reglas[i], estado, id_ficha))

    return jugadas


def bucle_saltar(jugadas, e, id_ficha):
    p = e.pos_fichas

    movs_posibles = reglas_saltar(e.tablero, p[id_ficha][0], p[id_ficha][1])

    if len(movs_posibles) != 0:
        for i in range(len(movs_posibles)):

            nuevo_e = saltar(movs_posibles[i], e, id_ficha)

            if not pos_repetida(jugadas, nuevo_e, id_ficha):
                jugadas.append(nuevo_e)
                bucle_saltar(jugadas, nuevo_e, id_ficha)

def pos_repetida(jugadas, nuevo_e, k):
    for i in range(len(jugadas)):
        xi, yi = jugadas[i].posicion_ficha(k)
        x, y = nuevo_e.posicion_ficha(k)
        if x == xi and y == yi:
            return True
    return False



def jugadas_saltar(estado, id_ficha):
    jugadas = [estado]
    bucle_saltar(jugadas, estado, id_ficha)
    jugadas.pop(0)
    return jugadas



def jugadas_posibles(ec):
    sucesores = []
    for k in range(n_fichas):
        estados_mover = jugadas_mover(ec.estado, k)
        for i in range(len(estados_mover)):
            nuevo_ec = EstadoComp(estados_mover[i], ec.g + 1, valor_h(estados_mover[i].pos_fichas), k)
            sucesores.append(nuevo_ec)

        estados_saltar = jugadas_saltar(ec.estado, k)
        for i in range(len(estados_saltar)):
            nuevo_ec = EstadoComp(estados_saltar[i], ec.g + 1, valor_h(estados_saltar[i].pos_fichas), k)
            sucesores.append(nuevo_ec)

    return sucesores



def seleccion_primero_el_mejor(ec):
    sucesores = jugadas_posibles(ec)
    sucesores_ordenados = sorted(sucesores, key=usa_f)

    return sucesores_ordenados[0]

def usa_f(ec):
    return ec.feval()


def estrategia_primero_el_mejor():
    global T, pos, n_movs_M, ficha_mov
    ec = EstadoComp(Estado(T, pos), n_movs_M, valor_h(pos), ficha_mov)
    nuevo_ec = seleccion_primero_el_mejor(ec)
    T = nuevo_ec.estado.tablero
    pos = nuevo_ec.estado.pos_fichas
    ficha_mov = nuevo_ec.ult_ficha_mov
    n_movs_M += 1


def fuera_tablero(x, y, tab):
    return not (0 <= x < len(tab) and 0 <= y < len(tab))


def direccion_mover(x, y, id_mov):
    tx = ty = 0

    if id_mov in movs:
        dx, dy = movs[id_mov]
        tx = x + dx
        ty = y + dy

    return tx, ty


def direccion_saltar(x, y, id_mov):
    tx = ty = 0

    if id_mov in movs:
        dx, dy = movs[id_mov]
        tx = x + 2 * dx
        ty = y + 2 * dy

    return tx, ty


def comprobar(x, y, tab, saltos_realizados):
    comp = {}

    for i in movs:
        dx, dy = movs[i]
        tx = x + dx
        ty = y + dy

        if not fuera_tablero(tx, ty, tab) and abs(tab[tx][ty]) == 1:
            tx = x + 2*dx
            ty = y + 2*dy

            if not fuera_tablero(tx, ty, tab) and tab[tx][ty] == 0:
                if (tx, ty) not in saltos_realizados:
                    comp[i] = (tx, ty)

    return comp



def continuar(x, y, tab, permitidos, visitados):
    jugada = comprobar(x, y, tab, visitados)

    if jugada:
        for i in jugada:
            visitados.append(jugada[i])


    if jugada:
        for i in jugada:
            x1, y1 = jugada[i]
            act_pos(x, y, x1, y1, tab)
            permitidos.append((x1, y1))
            continuar(x1, y1, tab, permitidos, visitados)

    return permitidos


def Comp_Meta(tab_actual, tab_meta,ficha):
    cont =0
    for i in range(len(tab_actual)):
        for j in range(len(tab_actual[i])):
            if(tab_actual[i][j]==tab_meta[i][j]==ficha):
                cont += 1
    return cont


def fin_juego():
    global FICHA_GANADORA
    if comp_tablero(T, T_Meta_M):
        FICHA_GANADORA = 'M'
    elif comp_tablero(T, T_Meta_H):
        FICHA_GANADORA = 'H'
        
        
def jugar_maquina():
    estrategia_primero_el_mejor()
    fin_juego()
    
def act_pos(x, y, x1, y1, tab):
    for i in range(len(tab)):
        if x == pos[i][0] and y == pos[i][1]:
            pos[i][0] = x1
            pos[i][1] = y1

    tab[x][y] = 0
    tab[x1][y1] = -1
    
def mover_ficha(x, y, tab):
    jug = []
    tablero = cp.deepcopy(tab)

    for i in movs:
        x1, y1 = direccion_mover(x, y, i)

        if fuera_tablero(x1, y1, tablero):
            continue

        elif abs(tablero[x1][y1]) == 1:

            x1, y1 = direccion_saltar(x, y, i)

            if fuera_tablero(x1, y1, tablero):
                continue

            elif abs(tablero[x1][y1]) == 1:
                continue

            else:
                jug += [(x1, y1)]
                act_pos(x, y, x1, y1, tablero)
                jug += continuar(x1, y1, tablero, [], [(x, y)])

        else:
            jug += [(x1, y1)]
            act_pos(x, y, x1, y1, tablero)

    return jug


def GetPath(Archivo):
    ruta_absoluta = os.path.join(os.getcwd(), Archivo)
    return ruta_absoluta

OpenSans = GetPath("OpenSans.ttf")
fonttt = GetPath("font.ttf")

def Botones():
    BotonPartida.dibujar(ventana_2)
    BotonJuego.dibujar(ventana_2)
    mouse_pos = pygame.mouse.get_pos()
    if BotonPartida.detectar_clic(mouse_pos):
        Posciones.append(mouse_pos)
    elif BotonJuego.detectar_clic(mouse_pos):
        Posciones.append(mouse_pos)
        
def Imprimir(titulo, t1,t2,x,y):
    a = texto.render(titulo,False, NEGRO)
    a = a.convert_alpha()
    a = pygame.transform.smoothscale(a, (t1, t2))
    ventana_2.blit(a,(x, y))
    
def ImprimirNegrita(titulo, t1,t2,x,y):
    a = Meta.render(titulo,False, NEGRO)
    a = a.convert_alpha()
    a = pygame.transform.smoothscale(a, (t1, t2))
    ventana_2.blit(a,(x, y))
    
def Contador():
    # Tiempo transcurrido
    elapsed_time = time.time() - start_time
    # Cálculo de las horas, minutos y segundos
    hours = int(elapsed_time / 3600)
    minutes = int((elapsed_time - (hours * 3600)) / 60)
    seconds = int(elapsed_time - (hours * 3600) - (minutes * 60))

    time_str = "{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds)

    time_surface = font.render(time_str, False, NEGRO)
    time_surface = time_surface.convert_alpha()
    scaled_surface = pygame.transform.smoothscale(time_surface, (95, 25))
    scaled_surface
    ventana_2.blit(scaled_surface, (98, 67))
    
def Botonesv2(event):
    global C_F1,C_F2,ventana_2,start_time,background,FICHA_GANADORA,jugadas,texto
    MENU_MOUSE_POS = pygame.mouse.get_pos()
    for boton in [Boton1,Boton2,Boton3,Boton4]:
        boton.changeColor(MENU_MOUSE_POS)
        boton.update(ventana_1)
    if event.type == pygame.MOUSEBUTTONDOWN:
        if Boton1.checkForInput(MENU_MOUSE_POS):
            pygame.display.quit()
            ventana_2 = pygame.display.set_mode((TAM_TABLERO, TAM_TABLERO))
            background = pygame.image.load(GetPath("TableroPor.png"))
            pygame.display.set_icon(icono)
            pygame.display.set_caption("Tablero de Esquinas Chinas")
            estado_inicial()
            jugadas=[]
            texto=pygame.font.Font(OpenSans, 150)
            FICHA_GANADORA = ''

            # Tiempo inicial
            start_time = time.time()
        if Boton2.checkForInput(MENU_MOUSE_POS):
            C_F1 = (0, 74, 173)   # Color Ficha 1
            C_F2 = (12, 192, 223) # Color Ficha 2
        if Boton3.checkForInput(MENU_MOUSE_POS):
            C_F1 = (12, 192, 223)   # Color Ficha 1
            C_F2 = (0, 74, 173)     # Color Ficha 2
        if Boton4.checkForInput(MENU_MOUSE_POS):
            pygame.quit()
            sys.exit()
            
def Jugadores():
    Imprimir("Jugador Computadora", 250,30,450,48)
    ImprimirNegrita("Meta: "+ str(Comp_Meta(T, T_Meta_M,1)), 70,25,540,85)
    Imprimir("Jugador Persona", 180,30,120,710)
    ImprimirNegrita("Meta: "+ str(Comp_Meta(T, T_Meta_H,-1)), 70,25,170,750)
    
def Tablero(fila, columna, evento):
    global Posciones, jugadas
    init=(fila,columna)
    if(init in jugadas and len(BAL)!=0):
        cuadrado = pygame.draw.rect(ventana_2, C_RES , pygame.Rect(columna * TAM_CASILLA+120,fila * TAM_CASILLA+120, TAM_CASILLA, TAM_CASILLA))
        mouse_pos = pygame.mouse.get_pos()
        if cuadrado.collidepoint(mouse_pos):
            Posciones.append(mouse_pos)
            if evento.type == pygame.MOUSEBUTTONDOWN:
                act_pos(x0, y0, fila, columna, T)
                jugadas=[]
                time.sleep(1)
                jugar_maquina()
    else:
        if((fila+columna)%2==0):
            pygame.draw.rect(ventana_2, BLANCO , pygame.Rect(columna * TAM_CASILLA+120,fila * TAM_CASILLA+120, TAM_CASILLA, TAM_CASILLA))
        else:
            pygame.draw.rect(ventana_2, GRIS , pygame.Rect(columna * TAM_CASILLA+120,fila * TAM_CASILLA+120, TAM_CASILLA, TAM_CASILLA))
    pygame.draw.rect(ventana_2, NEGRO , pygame.Rect(columna * TAM_CASILLA+120,fila * TAM_CASILLA+120, TAM_CASILLA, TAM_CASILLA),1)

def Fichas(fila, columna):
    global BAL, jugadas, Posciones,x0,y0
    init=T[fila][columna]
    if(init == 1):
        pygame.draw.circle(ventana_2, C_F2, (columna * TAM_CASILLA + TAM_CASILLA+ 145 // 2, fila * TAM_CASILLA + TAM_CASILLA+ 148 // 2), 34)
        
    elif(init == -1):
        circle_pos=[columna * TAM_CASILLA + TAM_CASILLA+ 145 // 2,fila * TAM_CASILLA + TAM_CASILLA+ 148 // 2]
        
        # Detectar si el mouse está encima del círculo
        mouse_pos = pygame.mouse.get_pos()
        distance = math.sqrt((mouse_pos[0] - circle_pos[0])**2 + (mouse_pos[1] - circle_pos[1])**2)
        
        if distance <= 34:
            # El mouse está encima del círculo, cambiar el color
            pygame.draw.circle(ventana_2, F_SOM, (circle_pos[0], circle_pos[1]), 34)

            Posciones.append(mouse_pos)
            if evento.type == pygame.MOUSEBUTTONDOWN:
                pygame.time.delay(100) # Espera 100 milisegundos (0.1 segundos)

                if len(BAL) == 0 :
                    jugadas = mover_ficha(fila, columna, T)
                    x0=fila
                    y0=columna
                    BAL.append(fila)
                else:
                    BAL=[]
        else:
            pygame.draw.circle(ventana_2, C_F1, (circle_pos[0], circle_pos[1]), 34)
            
def get_font(size): # Returns Press-Start-2P in the desired size
    return pygame.font.Font(fonttt, size)

class Boton:
    def __init__(self, x, y, w, h, color, texto):
        self.rect = pygame.Rect(x, y, w, h)
        self.color = color
        self.texto = texto
        self.font = pygame.font.Font(None, int(h * 0.6))

    def dibujar(self, superficie):
        pygame.draw.rect(superficie, self.color, self.rect, border_radius=4)
        texto = self.font.render(self.texto, True, BLANCO)
        text_rect = texto.get_rect(center=self.rect.center)
        superficie.blit(texto, text_rect)

    def detectar_clic(self, pos):
        return self.rect.collidepoint(pos)

    
class Confetti:
    def __init__(self):
        self.x = random.randint(0, TAM_TABLERO)
        self.y = random.randint(-TAM_TABLERO, -10)
        self.speed = random.randint(1, 5)
        self.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        self.size = random.randint(5, 10)

    def fall(self):
        self.y += self.speed
        if self.y > TAM_TABLERO:
            self.y = random.randint(-TAM_TABLERO, -10)

    def draw(self):
        pygame.draw.circle(ventana_2, self.color, (self.x, self.y), self.size)

    
class BotonV2():
    def __init__(self, image, pos, text_input, font, base_color, hovering_color):
        self.image = image
        self.x_pos = pos[0]
        self.y_pos = pos[1]
        self.font = font
        self.base_color, self.hovering_color = base_color, hovering_color
        self.text_input = text_input
        self.text = self.font.render(self.text_input, True, self.base_color)
        if self.image is None:
            self.image = self.text
        self.rect = self.image.get_rect(center=(self.x_pos, self.y_pos))
        self.text_rect = self.text.get_rect(center=(self.x_pos, self.y_pos))

    def update(self, screen):
        if self.image is not None:
            screen.blit(self.image, self.rect)
        screen.blit(self.text, self.text_rect)

    def checkForInput(self, position):
        if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
            return True
        return False

    def changeColor(self, position):
        if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
            self.text = self.font.render(self.text_input, True, self.hovering_color)
        else:
            self.text = self.font.render(self.text_input, True, self.base_color)
            
pygame.init()

# Musica
pygame.mixer.music.load(GetPath("Cancion_2.mp3"))

# Importar imagen
background = pygame.image.load(GetPath("fondo.png"))

# Cargar el icono
icono = pygame.image.load(GetPath("iconos.png"))

# Establecer el icono de la ventana
pygame.display.set_icon(icono)
pygame.display.set_caption("Tablero de Esquinas Chinas")

# Definir fuentes
font = pygame.font.SysFont(OpenSans, 250)
texto = pygame.font.SysFont(OpenSans, 250)
Meta = pygame.font.SysFont(OpenSans, 500)

# Reproducción en Bucle
pygame.mixer.music.play(-1)

# Botones
BotonPartida = Boton(522, 730, 170, 50, (45,0,153), 'Nueva Partida')
BotonJuego = Boton(330, 730, 170, 50, NEGRO, 'Salir del Juego')

Boton1 = BotonV2(image=pygame.image.load(GetPath("Options Rect.png")), pos=(350,250), 
                            text_input="JUGAR", font=get_font(35), base_color=TURQ, hovering_color="Black")
Boton2 = BotonV2(image=pygame.image.load(GetPath("1.png")), pos=(350, 350), 
                            text_input="   FICHA    ", font=get_font(35), base_color=C_F1, hovering_color="Black")
Boton3 = BotonV2(image=pygame.image.load(GetPath("2.png")), pos=(350, 450), 
                            text_input="   FICHA    ", font=get_font(35), base_color=C_F2, hovering_color="Black")
Boton4  = BotonV2(image=pygame.image.load(GetPath("Options Rect.png")), pos=(350, 550), 
                            text_input="SALIR", font=get_font(35), base_color=GRIS, hovering_color="Black")

for i in range(500):
    confetti_list.append(Confetti())

ventana_1 = pygame.display.set_mode((TAM_TABLERO, TAM_TABLERO))
ventana_2 = None

while True:
    # Manejo de eventos
    for evento in pygame.event.get():
        if evento.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    if ventana_2 is None:
        # Limpiar la ventana
        ventana_1.blit(background, (0, 0))
        iniciar=[]

        # Dibujar el tablero
        texto=pygame.font.Font(fonttt, 150)
        b = texto.render("Esquinas Chinas",False, NEGRO)
        b = b.convert_alpha()
        b = pygame.transform.smoothscale(b, (400, 110))
        ventana_1.blit(b,(150, 60))
        #Agregar los botones
        Botonesv2(evento)

        pygame.display.flip()
        pygame.time.delay(10)
    else:
        # Limpiar la ventana
        ventana_2.blit(background, (0, 0))
        iniciar=[]


        Posciones=[]
        Contador()
        Botones()
        Jugadores()

        if(FICHA_GANADORA!='M' and FICHA_GANADORA!='H'):
        # Dibujar el tablero
            for fila in range(NUM_FILAS):
                for columna in range(NUM_COLUMNAS):
                    Tablero(fila, columna,evento)
                    Fichas(fila, columna)

        else:
            if FICHA_GANADORA =='H':
                ventana_2.blit(imagen_victoria, (0,0))
                for confetti in confetti_list:
                    confetti.fall()
                    confetti.draw()
            else:
                ventana_2.blit(imagen_derrota, (0,0))

        mouse_pos = pygame.mouse.get_pos()

        if mouse_pos in Posciones:
            pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_HAND)
        else:
            pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_ARROW)

        if evento.type == pygame.MOUSEBUTTONDOWN:
            if BotonPartida.detectar_clic(pygame.mouse.get_pos()):
                pygame.display.quit()
                ventana_2=None
                ventana_1 = pygame.display.set_mode((TAM_TABLERO, TAM_TABLERO))
                background = pygame.image.load(GetPath("fondo.png"))
                pygame.display.set_icon(icono)
                pygame.display.set_caption("Tablero de Esquinas Chinas")
            if BotonJuego.detectar_clic(pygame.mouse.get_pos()):
                pygame.quit()

        # Actualizar la ventana
        pygame.display.flip()